In [1]:
import sys 
print(sys.version)

3.10.11 (main, Apr 20 2023, 19:02:41) [GCC 11.2.0]


In [2]:
!conda list transformers

# packages in environment at /home/bread/anaconda3/envs/py31011:
#
# Name                    Version                   Build  Channel
transformers              4.28.1                   pypi_0    pypi


In [3]:
!pwd

/home/bread/A_prc/transformer


# Chapter 4. 다중 언어 개체명 인식

지금까지는 NLP 작업을 하기 위해 트랜스포머를 영어 말뭉치에 적용

다국어를 사용하고 싶을 때는 다중 언어 트랜스포머 모델이 도움을 줌

BERT처럼 이런 모델은 사전 훈련 목표로 마스크드 언어 모델링을 사용하지만 100개 이상의 언어로 된 텍스트에서 동시에 훈련함

다중 언어 트랜스포머는 많은 언어로 된 대규모 말뭉치에서 사전 훈련해서 **제로샷 교차 언어 전이(Zero-shot cross-lingual transfer)** 가 가능함

  - 한 언어에서 미세 튜닝된 모델이 추가 훈련없이 다른 언어에 적용된다는 의미!
  
  - 이런 모델은 한 대화에서 둘 이상의 언어나 사투리를 바꾸는 '코드 스위칭(code switching)'에 적합함

이 장에서는 XLM-RoBERTa 트랜스포머 모델을 개체명 인식(NER: Named Entity Recognition)을 수행하도록 여러 언어에서 미세 튜닝하는 방법을 알아보겠음

1장에서 보았듯이 NER은 텍스트에서 사람, 조직, 위치 같은 개체명을 식별하는 일반적인 NLP 작업으로 다양한 애플리케이션에 사용함

  - 가령, 회사 문서에서 중요한 정보를 추출하거나, 검색 엔진의 품질을 높이거나, 말뭉치에서 구조적인 데이터베이스를 만드는데 사용함
  
이 장에서는 네 개의 공용어(영어가 언어를 이어주는 다리 역할을 하는 경우가 많습니다.)를 사용하며, 스위스에서 주로 활동하는 고객을 위해 NER을 수행한다고 가정

> **(NOTE)** **제로샷 전이(zero-shot transfer)** 또는 **제로샷 학습(zero-shot learning)** 은 일반적으로 한 레이블 집합에서 모델을 훈련한 다음 다른 레이블 집합에서 평가하는 작업을 의미함. 트랜스포머에서 제로샷 학습은 GPT-3 같은 언어 모델을 후속 작업에서 미세튜닝하지 않고 평가하는 상황을 의미하기도 함

## 4.1 데이터셋

이 장에서는 WikiANN 또는 PAN-X라 불리는 교차 언어 전이 평가(Cross-lingual TRansfer Evaluation of Multilingual Encoders(XTREME) 벤치 마크의 데이터를 사용함

이 데이터는 여러 언어의 위키피디아 문서로 구성되고 스위스에서 사용되는 독일어(62.9%), 프랑스어(22.9%), 이탈리아어(8.4%), 영어(5.8%) 네 개 언어로 작성됨

각 문서는 IOB2 포맷(Inside-Outside Beginning)으로 LOC(위치), PER(사람), ORG(조직) 태그가 표시되어 있음

이 포맷에서 B- 접두사는 개체명의 시작을 나타내고 동일한 개체명에 속해 연속되는 토큰에는 I- 접두사가 붙음.

  - 0 태그는 토큰이 어떤 개체명에도 속하지 않음을 나타냄
  
  - Jeff Dean is a computer scientist at Google in California 에 대한 IOB2 포맷으로 나타낸 시퀀스의 예
  
  | Tokens | Jeff | Dean  | is  | a  | computer  | scientist  | at   | Google  | in   | California  |
  |:-------|:-----|:------|:----|:---|:----------|:-----------|:-----|:--------|:-----|:------------|
  | **Tags**|B-PER |I-PER  | O   |O   |O          |O           |O     |B-ORG    |O     |B-LOC        |

XTREME에서 PAN-X 서브셋(subset) 중 하나를 로드하려면 load_dataset() 함수에 전달할 데이터셋 이름이 필요함

여러 서브셋이 포함된 데이터셋을 다룰 때 어떤 서브셋이 제공되는지 확인하려면 get_dataset_config_names() 함수를 호출

In [4]:
from datasets import get_dataset_config_names

xtreme_subsets = get_dataset_config_names("xtreme")
print(f"XTREME 서브셋 개수:  {len(xtreme_subsets)}")

XTREME 서브셋 개수:  183


서브셋이 많으므로 검색 범위를 좁혀 'PAN'으로 시작하는 서브셋을 찾음

In [5]:
panx_subsets = [s for s in xtreme_subsets if s.startswith("PAN")]
panx_subsets[:3]

['PAN-X.af', 'PAN-X.ar', 'PAN-X.bg']

-PAN-X 서브셋의 이름을 확인

ISO 639-1 언어 코드(https://en.wikipedia.org/wiki/List_of_ISO_639-1_codes)로 보이는 두 문자로 된 접미사가 있음

독일어 말뭉치를 로드하려면 다음처럼 load_dataset() 함수의 name 매개변수에 독일어(de)에 해당하는 코드 전달


In [6]:
from datasets import load_dataset

load_dataset("xtreme", name = "PAN-X.de")

Found cached dataset xtreme (/home/bread/.cache/huggingface/datasets/xtreme/PAN-X.de/1.0.0/29f5d57a48779f37ccb75cb8708d1095448aad0713b425bdc1ff9a4a128a56e4)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 20000
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 10000
    })
})

사실적인 스위스 말뭉치를 만들기 위해 스위스에서 사용되는 언어 비율로 PAN-X에서 독일어(de), 프랑스어(fr), 이탈리아어(it), 영어(en) 말뭉치를 샘플링

그러면, 언어가 불균형한 데이터셋이 만들어지는데 실제 데이터셋에서는 흔히 벌어지는 일

소수 언어에서 레이블링된 샘플을 구하려면 비용이 많이 들기도 함

소수 언어에 유창한 도메인 전문가가 드물기 때문

불균형한 이 데이터셋을 사용해 다중 언어 애플리케이션을 다룰 때 발생하는 일반적인 상황을 시뮬레이션하고, 모든 언어에서 작동하는 모델을 만드는 방법을 알아 봄

  - 각 언어를 추적하기 위해 파이썬 defaultdict 객체를 생성

  - 다음으로 언어 코드를 키로 하고 DatasetDict 타입의 PAN-X 말뭉치를 값으로 저장함

In [7]:
from collections import defaultdict
from datasets import DatasetDict

langs = ["de", "fr", "it", "en"]
fracs = [0.629, 0.229, 0.084, 0.059]

# 키가 없는 경우 DatasetDict를 반환
panx_ch = defaultdict(DatasetDict)

for lang, frac in zip(langs, fracs):
    # 다국어 말뭉치를 로드합니다.
    ds = load_dataset("xtreme", name = f"PAN-X.{lang}")
    # 각 분할을 언어 비율에 따라 다운샘플링하고 섞습니다. 
    for split in ds:
        panx_ch[lang][split] = (
            ds[split].shuffle(seed = 0).select(range(int(frac*ds[split].num_rows))))

Found cached dataset xtreme (/home/bread/.cache/huggingface/datasets/xtreme/PAN-X.de/1.0.0/29f5d57a48779f37ccb75cb8708d1095448aad0713b425bdc1ff9a4a128a56e4)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached shuffled indices for dataset at /home/bread/.cache/huggingface/datasets/xtreme/PAN-X.de/1.0.0/29f5d57a48779f37ccb75cb8708d1095448aad0713b425bdc1ff9a4a128a56e4/cache-e5ddf09f1ae095ec.arrow
Loading cached shuffled indices for dataset at /home/bread/.cache/huggingface/datasets/xtreme/PAN-X.de/1.0.0/29f5d57a48779f37ccb75cb8708d1095448aad0713b425bdc1ff9a4a128a56e4/cache-25e7e2dd003d0fa6.arrow
Loading cached shuffled indices for dataset at /home/bread/.cache/huggingface/datasets/xtreme/PAN-X.de/1.0.0/29f5d57a48779f37ccb75cb8708d1095448aad0713b425bdc1ff9a4a128a56e4/cache-73a95bc0accfea8b.arrow
Found cached dataset xtreme (/home/bread/.cache/huggingface/datasets/xtreme/PAN-X.fr/1.0.0/29f5d57a48779f37ccb75cb8708d1095448aad0713b425bdc1ff9a4a128a56e4)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached shuffled indices for dataset at /home/bread/.cache/huggingface/datasets/xtreme/PAN-X.fr/1.0.0/29f5d57a48779f37ccb75cb8708d1095448aad0713b425bdc1ff9a4a128a56e4/cache-6ff29513007ec78b.arrow
Loading cached shuffled indices for dataset at /home/bread/.cache/huggingface/datasets/xtreme/PAN-X.fr/1.0.0/29f5d57a48779f37ccb75cb8708d1095448aad0713b425bdc1ff9a4a128a56e4/cache-c5c9a4fc19dfd7d6.arrow
Loading cached shuffled indices for dataset at /home/bread/.cache/huggingface/datasets/xtreme/PAN-X.fr/1.0.0/29f5d57a48779f37ccb75cb8708d1095448aad0713b425bdc1ff9a4a128a56e4/cache-9711ab25936b81b7.arrow
Found cached dataset xtreme (/home/bread/.cache/huggingface/datasets/xtreme/PAN-X.it/1.0.0/29f5d57a48779f37ccb75cb8708d1095448aad0713b425bdc1ff9a4a128a56e4)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached shuffled indices for dataset at /home/bread/.cache/huggingface/datasets/xtreme/PAN-X.it/1.0.0/29f5d57a48779f37ccb75cb8708d1095448aad0713b425bdc1ff9a4a128a56e4/cache-daa9a1770078307c.arrow
Loading cached shuffled indices for dataset at /home/bread/.cache/huggingface/datasets/xtreme/PAN-X.it/1.0.0/29f5d57a48779f37ccb75cb8708d1095448aad0713b425bdc1ff9a4a128a56e4/cache-5e244c05031bab3c.arrow
Loading cached shuffled indices for dataset at /home/bread/.cache/huggingface/datasets/xtreme/PAN-X.it/1.0.0/29f5d57a48779f37ccb75cb8708d1095448aad0713b425bdc1ff9a4a128a56e4/cache-497ee15c12bff58d.arrow
Found cached dataset xtreme (/home/bread/.cache/huggingface/datasets/xtreme/PAN-X.en/1.0.0/29f5d57a48779f37ccb75cb8708d1095448aad0713b425bdc1ff9a4a128a56e4)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached shuffled indices for dataset at /home/bread/.cache/huggingface/datasets/xtreme/PAN-X.en/1.0.0/29f5d57a48779f37ccb75cb8708d1095448aad0713b425bdc1ff9a4a128a56e4/cache-757845faa9fa6949.arrow
Loading cached shuffled indices for dataset at /home/bread/.cache/huggingface/datasets/xtreme/PAN-X.en/1.0.0/29f5d57a48779f37ccb75cb8708d1095448aad0713b425bdc1ff9a4a128a56e4/cache-305cefc7ffa49fd9.arrow
Loading cached shuffled indices for dataset at /home/bread/.cache/huggingface/datasets/xtreme/PAN-X.en/1.0.0/29f5d57a48779f37ccb75cb8708d1095448aad0713b425bdc1ff9a4a128a56e4/cache-e5ec5e6ba7c1237d.arrow


데이터셋에 의도하지 않은 편향이 들어가지 않도록 shuffle() 메서드를 사용

select() 메서드는 fracs 값을 따라 각 말뭉치를 다운 샘플링합니다. 

Dataset.num_rows 속성으로 훈련 세트에 언어마다 얼마나 많은 샘플이 들어 있는지 확인

In [8]:
import pandas as pd

pd.DataFrame({lang: [panx_ch[lang]["train"].num_rows] for lang in langs},
            index = ["Number of training examples"])

,de,fr,it,en
Number of training examples,12580,4580,1680,1180


의도한 대로 독일어 샘플은 그 외 언어를 모두 합친 것보다 많음

이 데이터셋을 사용해 한 언어에서 미세 튜닝된 모델이 추가 훈련없이 다른 언어에 적용되는 제로샷 교차 언어 전이를 프랑스어, 이탈리아어, 영어에 수행

독일어 말뭉치에 있는 샘플 하나 확인

In [9]:
element = panx_ch["de"]["train"][0]
for key, value in element.items():
    print(f"{key}: {value}")

tokens: ['2.000', 'Einwohnern', 'an', 'der', 'Danziger', 'Bucht', 'in', 'der', 'polnischen', 'Woiwodschaft', 'Pommern', '.']
ner_tags: [0, 0, 0, 0, 5, 6, 0, 0, 5, 5, 6, 0]
langs: ['de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de']


앞서 본 Dataset 객체와 마찬가지로, 이 샘플의 키는 애로우(arrow) 테이블의 열 이름에 해당하고 값은 각 열에 있는 항목

특히 ner_tags 열은 각 개체명이 매핑된 클래스 ID


In [10]:
# 이해하기 쉽도록 LOC, PER, ORG 태그에 새로운 열 생성
# Dataset 객체는 각 열의 데이터 타입을 담은 features 속성을 가짐

for key, value in panx_ch["de"]["train"].features.items():
    print(f"{key}:{value}")

tokens:Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)
ner_tags:Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)
langs:Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)


sequence 클래스는 이 필드가 특성 리스트를 담고 있음을 나타냄

ner_tags의 경우 ClassLabel의 리스트 

In [11]:
# 훈련세트의 특성 확인

tags = panx_ch["de"]["train"].features["ner_tags"].feature
print(tags)

ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None)


2장에서 본 ClassLabel.int2str() 메서드를 사용해 각 태그의 클래스 이름을 담은 새로운 열(ner_tags_str)을 훈련 세트에 추가

map() 메서드에 하나의 dict를 반환하는 함수를 적용 

이 딕셔너리의 키는 새로운 열 이름이고 값은 태그 클래스 이름의 List


In [12]:
def create_tag_names(batch):
    return {"ner_tags_str": [tags.int2str(idx) for idx in batch["ner_tags"]]}

panx_de = panx_ch["de"].map(create_tag_names)

Loading cached processed dataset at /home/bread/.cache/huggingface/datasets/xtreme/PAN-X.de/1.0.0/29f5d57a48779f37ccb75cb8708d1095448aad0713b425bdc1ff9a4a128a56e4/cache-f3309bc5e9d28268.arrow
Loading cached processed dataset at /home/bread/.cache/huggingface/datasets/xtreme/PAN-X.de/1.0.0/29f5d57a48779f37ccb75cb8708d1095448aad0713b425bdc1ff9a4a128a56e4/cache-36ed4bdaca6e2a48.arrow
Loading cached processed dataset at /home/bread/.cache/huggingface/datasets/xtreme/PAN-X.de/1.0.0/29f5d57a48779f37ccb75cb8708d1095448aad0713b425bdc1ff9a4a128a56e4/cache-a08bda4878cb6bac.arrow


이제 태그를 읽기 좋게 변환함

훈련 세트에 있는 첫 번째 샘플의 토큰과 태그 이름을 나란히 출력

In [13]:
de_example = panx_de["train"][0]
pd.DataFrame([de_example["tokens"], de_example["ner_tags_str"]], ['Tokens', 'Tags'])

,0,1,2,3,4,5,6,7,8,9,10,11
Tokens,2.000,Einwohnern,an,der,Danziger,Bucht,in,der,polnischen,Woiwodschaft,Pommern,.
Tags,O,O,O,O,B-LOC,I-LOC,O,O,B-LOC,B-LOC,I-LOC,O


'2,000 Einwohnern an der Danziger Bucht in der polnischen Woiwodschaft Pommern'는 영어로 '2,000 inhabitants at the Gdansk Bay in the Polish voivodeship of Pomerania'임

그단스크만(Gdansk Bay)은 발트해(Baltic Sea)에 있는 만이고 voivodeship은 폴란드의 행정구역을 의미하기 때문에 LOC 태그가 붙은 것이 이해됨

태크가 불균형하게 부여되지 않았는지 확인하기 위해 각 분할에서 개체명의 빈도를 계산

In [14]:
from collections import Counter

split2freqs = defaultdict(Counter)

for split, dataset in panx_de.items():
    for row in dataset["ner_tags_str"]:
        for tag in row:
            if tag.startswith("B"):
                tag_type = tag.split("-")[1]
                split2freqs[split][tag_type] += 1
                
pd.DataFrame.from_dict(split2freqs, orient = "index")

,LOC,ORG,PER
train,6186,5366,5810
validation,3172,2683,2893
test,3180,2573,3071


PER, LOC, ORG 빈도 분포가 대체로 각 분할에서 동일

이 검증 세트와 테스트 세트는 NER 태그의 일반화 능력을 평가하는 데 적절하다고 판단

다음으로 인기 있는 다중 언어 트랜스포머를 몇 가지 살펴보고 **NER** (Named Entity Recognition) 작업을 어떻게 적용하는 지 확인

## 4.2 다중 언어 트랜스포머

다중 언어 트랜스포머의 훈련 과정과 아키텍처는 단일 언어 트랜스포머와 비슷

다만 사전 훈련에 사용하는 말뭉치가 여러 언어의 문서로 구성될 뿐

언어의 차이에 대한 정보가 명시적으로 제공되지 않아도 이렇게 구축한 언어 표현이 여러 언어의 다양한 후속 작업에 쉽게 일반화 

교차 언어 전이를 수행하는 이런 능력이 경우에 따라 단일 언어 모델과 비슷한 결과를 내므로 언어마다 별도의 모델을 훈련할 필요가 없음


NER에 대한 교차 언어 전이의 과정을 측정하기 위해 CoNL-2002(https://huggingface.co/datasets/conll2002)와 CoNLL-2003(https://huggingface.co/datasets/conll2003) 데이터 셋이 영어, 네덜란드어, 스페인어, 독일어를 위한 벤치마크로 많이 사용됨

이 벤치마크는 PAN-X와 동일하게 개체명이 LOC, PER, ORG 태그로 분류된 뉴스 기사로 구성됨

하지만 이 세 카테고리에 속하지 않는 개체명을 위해 MISC 레이블이 추가됨

다중 언어 트랜스포머 모델은 일반적으로 세 가지 방식으로 평가함

**(en)** 영어 훈련 데이터에서 미세 튜닝한 다음에 각 언어의 테스트 세트에서 평가

**(each)** 언어별 성능을 측정하기 위해 단일 언어의 테스트 세트에서 미세 튜닝하고 평가함

**(all)** 모든 훈련 데이터에서 미세 튜닝해 각 언어의 테스트 세트에서 평가함



NER 작업에 비슷한 평가 전략을 적용

ㅇ 먼저 평가할 모델 선택
 
  - 초기 다중 언어 트랜스포머 중 하나는 mBERT
  
  - 사전 훈련 목표가 BERT의 아키텍처와 동일하지만 사전 훈련 말뭉치에 다양한 언어의 위키피디아 문서가 추가되었고,
  
  - 이후 XLM-RoBERTAa(약칭 XLM-R)로 대체됨
  
ㅇ XLM-R은 사전 훈련 목표로 100개 언어에 대한 MLM만 사용함

  - 하지만 이전 모델에 비해 사전 훈련 말뭉치가 매우 큼
  
  - 각 언어의 위키피디아 덤프와 2.5 테라바이트의 커먼 크롤(common crawl) 웹 데이터를 사용
  
  - 이 말뭉치는 초기 모델에서 사용한 데이터보다 몇 배가 더 크며 위키피디아 문서가 적은 버마어(Bumese)나 스와힐리어 같은 희소 언어에 대한 성능을 크게 높임
  
ㅇ 모델명에 RoBERTa가 붙은 이유는 사전 훈련 방식이 단일 언어 모델 RoBERTa와 같기 때문

  - RoBERTa의 개발자는 BERT를 여러 측면에서 향상시킴
  
  - 특히 다음 문장 예측 작업을 제거하며 BERT의 성능을 크게 높임
  
  - XLM-R은 XLM에서 사용하는 임베딩도 제거하고 SentencePiece를 사용해 원시 텍스트를 직접 토큰화 함
  
ㅇ XLM-R이 다중 언어라는 특징 외에 두 모델의 큰 차이점은 어휘 사전의 크기 

  - 사용하는 토큰의 개수가 RoBERTa는 55,000개지만 XLM-R은 250,000개

## 4.3 XLM-R 토큰화

XLM-R은 다국어 NLU 작업에 잘 맞는 모델

WordPiece 토크나이저 대신 100개 언어의 텍스트에서 훈련된 SentencePiece라는 토크나이저 사용

SentencePiece를 WordPiece와 비교하기 위해서 허깅페이스 트랜스포머스로 BERT와 XLM-R의 토크나이저를 로드함

In [15]:
from transformers import AutoTokenizer

bert_model_name = "bert-base-cased"
xlmr_model_name = "xlm-roberta-base"
bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
xlmr_tokenizer = AutoTokenizer.from_pretrained(xlmr_model_name)

짧은 텍스트 시퀀스르 인코딩해서 각 모델이 사전 훈련 동안에 사용하는 특수 토큰을 확인

In [16]:
text = "Jack Sparrow loves New York!"

bert_tokens = bert_tokenizer(text).tokens()
xlmr_tokens = xlmr_tokenizer(text).tokens()

In [17]:
print(bert_tokens)

['[CLS]', 'Jack', 'Spa', '##rrow', 'loves', 'New', 'York', '!', '[SEP]']


In [18]:
# 책에 나와 있는 표를 만들기 위해 shallow copy 실시

import copy

bert_tokens_add = copy.copy(bert_tokens)
bert_tokens_add.append("None")

print(bert_tokens_add)

['[CLS]', 'Jack', 'Spa', '##rrow', 'loves', 'New', 'York', '!', '[SEP]', 'None']


In [19]:
print(xlmr_tokens)

['<s>', '▁Jack', '▁Spar', 'row', '▁love', 's', '▁New', '▁York', '!', '</s>']


In [20]:
pd.DataFrame({'bert_tokens' : bert_tokens_add, 'xlmr_tokens' : xlmr_tokens}).transpose() 

,0,1,2,3,4,5,6,7,8,9
bert_tokens,[CLS],Jack,Spa,##rrow,loves,New,York,!,[SEP],None
xlmr_tokens,<s>,▁Jack,▁Spar,row,▁love,s,▁New,▁York,!,</s>


In [21]:
print(bert_tokens)

['[CLS]', 'Jack', 'Spa', '##rrow', 'loves', 'New', 'York', '!', '[SEP]']


BERT가 문장 분류 작업에 사용하는 `[CLS]`와 `[SEP]` 토큰 대신에, XLM-R은 `<s>`와 `</s>`를 사용해 시퀀스의 처음과 끝을 표시

  - 토큰화 마지막 단계에 추가됨

### 4.3.1 토큰화 파이프 라인

지금까지 토큰화를 다룰 때 문자열을 모델에 주입할 정수로 변환하는 연산으로만 여겼는데, 전적으로 옳은 것은 아님

자세히 들여다보면, 실제 전체 처리 파이프라인은 다음 그림처럼 네 개의 단계로 구성됨

> 정규화 > 사전 토큰화 > 토크나이저 모델 > 사후 처리

![그림4-1](image/chapter04_tokenizer-pipeline.png)

"Jack Sparrow loves New York!" 이란 예시 문장으로 각 처리 단계를 자세히 알아보고 그 효과를 확인

**정규화**

이 단계는 원시 문자열을 더 '깨끗하게' 만들기 위해 적용하는 일련의 연산에 해당함

대개 이런 연산에는 공백과 악센트가 붙은 문자를 제거하는 작업 등이 포함됨. 유니코드 정규화(https://unicode.org/reports/tr15/)는 많은 토크나이저에서 적용하는 또 다른 일반적인 정규화 연산이며, 같은 문자를 쓰는 여러 가지 방식을 처리함

같은 문자열(즉, 같은 추상 문자의 시퀀스)의 두 버전이 다르게 표시될 수 있음

NFC, NFD, NFKC, NFKD 같은 유니코드 정규화 방식은 같은 문자를 쓰는 여러 방식을 표준 형식을 대체함

  - NFC: Normalization Form Canonical Composition
  - NFD: Normalization Form Canonical Decomposition
  - NFKC: Normalization Form Compatibility Composition
  - NFKD: Normalization Form Compatibility Decomposition

(한글과 유니코드 참고문헌: https://gist.github.com/Pusnow/aa865fa21f9557fa58d691a8b79f8a6d)

정규화의 또 다른 방식은 소문자로 바꾸는 것.

모델이 소문자만 받을 것으로 기대한다면 이 기법을 사용해 어휘사전의 크기를 줄일 수 있음 -> tolower() 함수 같은 방법

  - 예시 문장은 "jack sparrow loves new york!" 형태가 됨

**사전 토큰화(pretokenization)**

이 단계는 텍스트를 더 작은 객체로 분할하며 훈련 마지막에 생성되는 토큰의 상한선을 제공함

사전 토큰화가 텍스트를 단어로 분할하고 최종 토큰은 이 단어의 일부가 된다고 생각하면 좋음

영어, 독일어, 많은 인도-유럽어족의 언어에서 문자열이 일반적으로 공백과 구두점을 기준 삼아 단어로 분할됨

예시 문장은 이 단계에서 `["jack", "sparrow", "loves", "new", "york", "!"]`로 변환됨

이런 단어는 파이프라인 다음 단계에서 **BPE(Byte-Pair Encoding)** 나 **유니그램(unigram)** 알고리즘을 사용해 부분단어(subword)로 분할하기 더 쉬움

하지만 단어로 분할하는 것이 항상 간단하고 확실한 연산인 것은 아님. 게다가 타당성 있는 연산도 아님

예를 들어 중국어, 일본어, 한국어 같은 언어에서 인도-유럽어족 단어와 같은 의미 단위로 기호를 모으면 똑같이 유효한 여러 그룹이 만들어지는 비결정적 연산이 됨

이 경우 텍스트를 사전 토큰화하는 대신 언어별 사전 토큰화 라이브러리를 사용하는 것이 좋음

**토크나이저 모델**

입력 텍스트 정규화와 사전 토큰화를 수행하고 난 후 토크나이저를 사용해 부분단어 분할 모델을 단어에 적용

토크나이저는 파이프라인에서 말뭉치로 훈련이 필요한 부분

(그렇지 않고 사전 훈련된 토크나이저를 사용한다면 이미 훈련된 상태로 볼 수 있음)

이 모델은 단어를 부분단어로 나눠 어휘사전의 크기와 OOV 토큰의 개수를 줄이는 역할을 함

부분 단어 토큰화 알고리즘에는 BPE, 유니그램, WordPiece 등이 있음. 

  - 예시 문장에 토크나이저 모델을 적용하면 `[jack, spa, rrow, loves, new, york, !]` 같이 됨
  
이 시점부터는 더 이상 문자열 리스트가 아닌 정수(입력ID) 리스트를 가지게 됨

책에서는 이해하기 쉽게 단어는 그대로 두고 따옴표를 삭제해 이런 정수 변환을 나타냄

**사후 처리**

토큰화 파이프라인의 마지막 단계로, 토큰 리스트에 부가적인 변환을 적용할 때도 있음

가령 입력 토큰 인덱스의 시퀀스 처음과 끝에 특수 토큰을 추가하는 경우

  - BERT 스타일의 토크나이저는 분류 토큰(CLS)과 분할 토큰(SEP)을 추가해 토큰 리스트는 `[CLS, jack, spa, rrow, loves, new, york, !, SEP]` 형태가 됨
  
  - 그 다음 모델에 이 시퀀스(책에 쓰인 토큰이 아니라 정수의 시퀀스)를 주입함
  
  > *CLS(classification token), SEP(seperation token)*

XLM-R과 BERT 비교로 돌아가면, SentencePiece는 사후 처리 단계에서 `[CLS]`와 `[SEP]` 대신 `<s>`와 `</s>`를 추가함
(편의상 그림 설명에는 계속 [CLS]와 [SEP] 사용)

그럼 SentencePiece 토크나이저가 특별한 이유를 알아보자

### 4.3.2 SentencePiece 토크나이저

SentencePiece 토크나이저는 유니그램이라는 부분단어 분할 방식을 기반으로 각 입력 텍스트를 유니코드 문자 시퀀스로 인코딩함


**(SentencePiece)** 의 특징

ㅇ SentencePiece가 악센트, 구두점에 대해 몰라도 되므로 이 특징은 다국어 말뭉치에 특히 유용

  - 일본어 등의 많은 언어에 공백 문자가 있지 않다는 사실도 몰라도 됨
  
ㅇ 공백 문자가 유니코드 기호 U+2581 또는 아래 1/4 블록 문자(lower one quater block character)라고도 하는 `_`할당된다는 것

  - 따라서 SentencePiece는 언어별 사전 토크나이저에 의존하지 않고 정확하게 시퀀스를 복원함

**비교**

이전 절의 예시 문장을 다시 보면 WordPiece는 'York'와 '!' 사이에 공백이 없다는 정보를 잃어버림. 이와 달리 SentencePiece는 토큰화된 텍스트에 공백을 보존하기 때문에 정확하게 원시 텍스트로 다시 변환됨



In [22]:
"".join(xlmr_tokens).replace(u"\u2581"," ")

'<s> Jack Sparrow loves New York!</s>'

이제 SentencePiece의 작동 원리를 이해했으니 간단한 샘플을 NER(Named Entity Recognition)에 적합한 형태로 인코딩하는 방법을 알아보겠음

먼저, 토큰 분류 헤드와 함게 사전 훈련된 모델을 로드

하지만 이 헤드를 허깅페이스 트랜스포머스에서 로드하지 않고 직접 작성 (허깅페이스 트랜스포머스 API를 자세히 살펴보면 모든 작업을 단 몇 단계로 처리할 수 있음)

## 4.4 개체명 인식을 위한 트랜스포머

2장에서 보았듯이, 텍스트 분류를 위해 BERT는 특수 토큰(CLS)로 전체 텍스트 시퀀스를 표현함. 아래 그림처럼 이 표현을 완전 연결 또는 밀집 층에 통과시켜 이산적인 레이블 값을 출력함

> **`[그림4-2]`** 시퀀스 분류를 위해 인코더 기반 트랜스포머 미세 튜닝
>
> ![그림4-2](image/chapter04_clf-architecture.png)

BERT와 그 외 인코더 기반 트랜스포머는 NER 작업에 비슷한 방식을 사용함

다만 모든 입력 토큰의 표현이 완전 연결 층에 주입되어 해당 토큰의 개체명을 인식함

이런 이유로 NER을 종종 토큰 분류 작업으로 생각하기도 함

> **`[그림 4-3]`** (전체 과정) 개체명 인식을 위해 인코더 기반 트랜스포머 미세 튜닝
>
> ![그림4-3](image/chapter04_ner-architecture.png)

하지만 토큰 분류 작업에서 부분단어를 어떻게 처리해야 할까? 예를 들어 [그림4-3]에서 이름 'Christina'는 부분단어 'Chr'와 '##ista'로 토큰화 됨. 이 중 어느 단어에 (아니면 두 단어 모두에) B-PER 레이블을 할당해야 하는가?

BERT 논문에서 저자들은 이 레이블을 첫 번째 부분단어(이 예에서 'Chr')에 할당하고 이어지는 부분단어(이 예에서 '##ista')는 무시함

  - 여기에서 이런 방식을 사용하고 무시한 부분 단어를 IGN으로 표시
  
  - 나중에 후처리 단계에서 첫 번째 부분단어의 예측 레이블을 후속 부분 단어로 쉽게 전파할 수 있음
  
  - '##ista' 부분 단어의 표현에 B-PER 레이블을 할당할 경우 IOB2 포맷을 위반
  
    + 다행히 XLM-R 아키텍처는 BERT와 동일한 RoBERTa를 기반으로 하므로 BERT에서 본 모든 아키텍처 특징이 XLM-R에도 있음

## 4.5 트랜스포머 모델 클래스

허깅페이스 트랜스포머스는 아키텍처와 작업마다 전용 클래스를 제공함

작업에 연관된 모델 클래스 이름은 `<ModelName>For<Task>` 형식을 따름

  - AutoModel 클래스를 사용하는 경우 `AutoModelFor<Task>`와 같음

허깅페이스 트랜스포머스는 기존 모델을 득정 작업에 맞춰 쉽게 확장 가능하도록 설계됨

사전 훈련된 모델에서 가중치를 로드하고 작업에 특화된 헬퍼 함수를 사용

아주 조금만 수고하면 특정 용도의 사용자 정의 모델을 만들 수 있음

### 4.5.1 바디와 헤드

허깅페이스 트랜스포머스의 다재다능한 능력은 `바디(body)`와 `헤드(head)`로 나뉜 모델 구조에서 나옴

  - 이 구조는 사전 훈련된 모델을 후속 작업에 맞게 바꿀 때 이미 봄
  
  - **(헤드)** 모델의 마지막 층이 후속 작업에 맞는 층으로 바뀌는데, 이 마지막 층이 모델의 헤드이며 작업에 특화됨

  - **(바디)** 모델의 나머지 부분을 바디라 함. 여기에는 작업에 특화되지 않은 토큰 임베딩과 트랜스포머 층이 포함됨
  
  - 이 구조는 허깅 페이스 트랜스포머 코드에도 반영됨
  
모델의 바디는 `BertModel` 또는 `GPT2Model` 같은 클래스로 구현되며 (바디에 있는) 마지막 층의 은닉 상태를 반환함

BertForMaskedLM 또는 BertForSequenceClassification 같이 작업에 특화된 모델은 `[그림 4-4]`처럼 베이스 모델을 사용하고 마지막 은닉 상태 위에 필요한 헤드를 추가함

> ![그림 4-4](image/chapter04_bert-body-head.png)
> 
> [그림 4-4] `BertModel` 클래스는 모델의 바디만 포함하고, `BertFor<Task>` 클래스는 바디와 작업 전용 헤드를 연결한 것

### 4.5.2 토큰 분류를 위한 사용자 정의 모델 만들기

XLM-R에 대한 사용자 정의 토큰 분류 헤드를 만드는 예제를 살펴봄

XLM-R은 RoBERTa와 모델 구조가 동일, RoBERTa를 베이스 모델로 사용하되 특화된 설정을 추가

먼저, XLM-R NER 태그를 표현할 데이터 구조가 필요함

  - 추측컨데 모델을 초기화할 설정 객체와 출력을 생성할 forward() 함수가 필요

In [23]:
import torch.nn as nn
from transformers import XLMRobertaConfig
from transformers.modeling_outputs import TokenClassifierOutput
from transformers.models.roberta.modeling_roberta import RobertaModel
from transformers.models.roberta.modeling_roberta import RobertaPreTrainedModel

class XLMRobertaForTokenClassification(RobertaPreTrainedModel):
    config_class = XLMRobertaConfig
    
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        # 모델 바디를 로드합니다. 
        self.roberta = RobertaModel(config, add_pooling_layer = False)
        # 토큰 분류 헤드를 준비
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        # 가중치를 로드하고 초기화 합니다. 
        self.init_weights()
        
    def forward(self, input_ids = None, attention_mask = None, token_type_ids = None, labels = None, **kwargs):
        
        # 모델 바디를 사용해 인코더 표현을 얻습니다.
        outputs = self.roberta(input_ids, attention_mask = attention_mask, token_type_ids = token_type_ids, **kwargs)
        
        # 인코더 표현을 헤드에 통과시킵니다. 
        sequence_output = self.dropout(outputs[0])
        logits = self.classifier(sequence_output)
        
        # 손실을 계산합니다. 
        loss = None
        
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            
        # 모델 출력 객체를 반환합니다. 
        return TokenClassifierOutput(loss = loss, 
                                     logits = logits, 
                                     hidden_states = outputs.hidden_states, 
                                     attentions = outputs.attentions)       

2023-05-23 19:07:26.093630: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-23 19:07:26.296596: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-23 19:07:26.872833: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


config_class는 새로운 모델을 초기화할 때 표준 XLM-R 설정을 사용하도록 도와줌

  - config_class는 RobertaPreTrainedModel의 부모 클래스 PreTrainedModel의 클래스 변수
  
  - 이 변수에 모델 구조로 사용할 설정 클래스(PreTrainedConfig의 서브 클래스)를 지정함
  
super() 메서드로 RobertaPreTrainedModel 클래스의 초기화 함수를 호출함

  - 이 추상 클래스는 사전 훈련된 가중치의 초기화나 로딩을 처리함
  
그 다음 모델 바디 RobertaModel을 로드하고 드롭아웃층과 피드 포워드 층으로 구성된 분류 헤드를 추가함

add_pooling_layer = False로 지정해서 `[CLS]` 토큰에 해당하는 은닉 상태 외에 모든 은닉 상태가 반환되도록 함

  - add_pooling_layer의 기본값은 True로 첫 번째 토큰의 은닉 상태만 밀집 층과 활성화 함수에 통과시켜 출력함

마지막으로 RobertaPreTrainedModel 클래스에서 상속된 init_weights() 메서드를 호출해 가중치를 초기화 함

  - init_weights() 메서드는 PreTrainedModel 클래스에 정의되며 RobertaPreTrainedModel 클래스에 정의된 init_weights() 메서드를 사용해 층의 가중치를 초기화함. init_weights()를 직접 호출하는 대신 이 메서드 호출을 담당하는 post_init() 메서드를 사용하는 것이 좋음

### 4.5.3 사용자 정의 모델 로드하기

토큰 분류 모델을 로드할 준비를 마침

모델 이름 외에 몇 가지 정보를 추가해야 함

각 개체명을 레이블링하는데 사용할 태그, 각 태그를 ID로 매핑하는 딕셔너리와 그 반대로 매핑하는 디셔너리

이런 정보는 모두 ClassLabel 객체인 tags 변수의 names 속성을 사용해 매핑함

In [24]:
index2tag = {idx: tag for idx, tag in enumerate(tags.names)}
tag2index = {tag: idx for idx, tag in enumerate(tags.names)}

3장에서 본 AutoConfig 객체에 이런 매핑과 tags.num_classes 속성을 저장함

from_pretrained() 메서드의 키워드 매개변수로 전달해 기본값을 오버라이드 함


In [25]:
from transformers import AutoConfig

xlmr_config = AutoConfig.from_pretrained(xlmr_model_name, 
                                         num_labels = tags.num_classes, 
                                         id2label = index2tag, 
                                         label2id = tag2index)

AutoConfig 클래스는 모델 구조의 청사진을 가짐

AutoModel.from_pretrained(model_ckpt)로 모델을 로드할 때 모델에 연관된 설정 파일이 자동으로 다운로드 됨

  - xlmr_config를 따로 전달하지 않으면, config_class 변수에 지정된 XLMRobertaConfig 클래스의 from_pretrained() 메서드를 호출해 (BERT 같은) 기본값이 지정된 설정 객체를 생성함
  
  
  - 하지만 클래스 개수나 레이블 이름 등을 수정하고 싶다면 커스터마이징하려는 매개변수로 이 설정 파일을 먼저 로드해야 함

이제 config 매개변수를 추가한 다음, 이전처럼 from_pretrained() 함수를 사용해 모델 가중치를 로드함

모델 클래스에 사전 훈련된 가중치를 로드하는 코드를 작성하지 않았지만, RobertaPreTrainedModel을 상속했으니 다음과 같이 로드할 수 있음

In [26]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
xlmr_model = (XLMRobertaForTokenClassification.from_pretrained(xlmr_model_name, config = xlmr_config).to(device))

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForTokenClassification: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'roberta.e

토크나이저와 모델을 바르게 초기화했는지 빠르게 확인하기 위해 개체명을 알고 있는 작은 시퀀스의 예측 결과를 확인

In [27]:
# 텍스트를 입력 ID로 변환

input_ids = xlmr_tokenizer.encode(text, return_tensors = "pt")
pd.DataFrame([xlmr_tokens, input_ids[0].numpy()], index = ["Tokens", "Input IDs"])

,0,1,2,3,4,5,6,7,8,9
Tokens,<s>,▁Jack,▁Spar,row,▁love,s,▁New,▁York,!,</s>
Input IDs,0,21763,37456,15555,5161,7,2356,5753,38,2


시작 `<s>`와 `<\s>` 토큰 ID는 각각 0과 2

마지막으로 모델에 입력 ID를 전달하고 argmax 함수로 토큰마다 확률이 가장 높은 클래스를 선택해 예측


In [28]:
outputs = xlmr_model(input_ids.to(device)).logits
predictions = torch.argmax(outputs, dim = -1)
print(f"시퀀스에 있는 토큰 개수: {len(xlmr_tokens)}" )
print(f"출력 크기: {outputs.shape}")

시퀀스에 있는 토큰 개수: 10
출력 크기: torch.Size([1, 10, 7])


로짓의 크기는 `[batch_size, num_tokens, num_tags]`

즉, 각 토큰에는 일곱개의 NER 태그와 로짓이 주어짐 

사전 훈련된 모델의 예측 결과를 태그 이름으로 바꿔 토근과 함께 확인함

In [29]:
preds = [tags.names[p] for p in predictions[0].cpu().numpy()]
pd.DataFrame([xlmr_tokens, preds], index = ["Tokens", "Tags"])

,0,1,2,3,4,5,6,7,8,9
Tokens,<s>,▁Jack,▁Spar,row,▁love,s,▁New,▁York,!,</s>
Tags,B-PER,B-PER,B-PER,B-PER,B-PER,B-PER,B-PER,B-PER,B-PER,B-PER


예상한 대로 랜덤한 가중치를 가진 토큰 분류 층은 개선할 여지가 많음 

레이블링 된 데이터로 미세 튜닝해서 결과를 더 좋게 만들어 봄 

훈련을 시작하기 전, 나중을 위해 앞의 과정을 하나의 헬퍼 함수로 만들어 봄

In [30]:
def tag_text(text, tags, model, tokenizer):
    # 토큰을 준비함
    tokens = tokenizer(text).tokens()
    # 시퀀스를 입력 ID로 인코딩 함
    input_ids = xlmr_tokenizer(text, return_tensors = "pt").input_ids.to(device)
    # 가능한 일곱 개의 클래스에 대한 로짓을 출력함
    outputs = model(input_ids)[0]
    # argmax 함수로 토큰마다 가장 가능성이 높은 클래스를 선택함
    predictions = torch.argmax(outputs, dim = 2)
    # 데이터 프레임으로 변환함
    preds = [tags.names[p] for p in predictions[0].cpu().numpy()]
    return pd.DataFrame([tokens, preds], index = ["Tokens", "Tags"])

## 4.6 NER 작업을 위한 텍스트 토큰화 하기

하나의 샘플을 인코딩하고 처리하는 토크나이저와 모델을 준비했으므로 다음 단계는 미세 튜닝을 위해 XLM-R 모델에 전달할 전체 데이터셋을 토큰화하는 것

2장에서 보았듯이 허깅페이스 데이터 셋은 map() 연산으로 Dataset 객체를 빠르게 토크놯함

이를 위해 다음처럼 간단한 시그니처(signature)를 가진 함수를 정의

```python

function(examples: Dict[str, List]) -> Dict[str, List]

```

여기서 examples는 Dataset의 슬라이스 하나에 해당. 예를 들면 `panx_de['train'][:10]` 

XLM-R 토크나이저는 모델 입력을 위해 입력ID를 반환함

따라서 반환된 결과에 어텐션 마스크와 각 토큰에 어떤 NER 태그가 연관됐는지 인코딩하는 레이블 ID를 추가 

허깅페이스 트랜스포커스 문서(https://huggingface.co/docs/transformers/tasks/token_classification)에 언급된 방법을 따라 독일어 샘플 문장 하나를 사용해 어떻게 하는 지 알아 봄

먼저 평범한 리스트로 단어와 태그를 준비함

In [37]:
words, labels = de_example["tokens"], de_example["ner_tags"]

그다음 각 단어를 토큰화합니다. 토크나이저에 is_split_into_words 매개변수를 사용해 입력 문장이 이미 단어로 나뉘어 졌다는 사실을 전함

In [38]:
tokenized_input = xlmr_tokenizer(de_example["tokens"], is_split_into_words = True)
tokens = xlmr_tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
pd.DataFrame([tokens], index = ["Tokens"])

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
Tokens,<s>,▁2.000,▁Einwohner,n,▁an,▁der,▁Dan,zi,ger,▁Buch,...,▁Wo,i,wod,schaft,▁Po,mmer,n,▁,.,</s>


이 예에서 토크나이저는 'Einwohnern'을 두 개의 부분단어 `'_Einwohner'와 'n'`으로 나눔

`'_Einwohner'` 만 B-LOC 레이블에 연결한다는 규칙을 따라야 하므로 첫 번째 부분단어 이후의 부분단어는 마스킹함

다행히 tokenized_input 객체에는 이 작업에 도움이 되는 word_ids() 메서드가 있음

In [39]:
word_ids = tokenized_input.word_ids()
pd.DataFrame([tokens, word_ids], index = ["Tokens", "Word IDs"])

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
Tokens,<s>,▁2.000,▁Einwohner,n,▁an,▁der,▁Dan,zi,ger,▁Buch,...,▁Wo,i,wod,schaft,▁Po,mmer,n,▁,.,</s>
Word IDs,None,0,1,1,2,3,4,4,4,5,...,9,9,9,9,10,10,10,11,11,None


word_ids는 각 부분단어를 words 리스트에 있는 해당 단어의 인덱스에 매핑

따라서 첫 번째 부분 단어 `'_2,000'`는 인덱스 0 이고 `'_Einwohner'와 'n'`는 인덱스 1에 할당됨

`<s>`, `</s>` 같은 특수 토큰은 None에 매핑함

이런 특수 토큰과 부분단어의 레이블을 -100으로 설정해 훈련하는 동안 마스킹 처리함

In [40]:
previous_word_idx = None
label_ids = []

for word_idx in word_ids:
    if word_idx is None or word_idx == previous_word_idx:
        label_ids.append(-100)
    elif word_idx != previous_word_idx:
        label_ids.append(labels[word_idx])
    previous_word_idx = word_idx
    
labels = [index2tag[l] if l != -100 else "IGN" for l in label_ids]
index = ["Tokens", "Word IDs", "Label IDs", "Labels"]

pd.DataFrame([tokens, word_ids, label_ids, labels], index = index)

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
Tokens,<s>,▁2.000,▁Einwohner,n,▁an,▁der,▁Dan,zi,ger,▁Buch,...,▁Wo,i,wod,schaft,▁Po,mmer,n,▁,.,</s>
Word IDs,None,0,1,1,2,3,4,4,4,5,...,9,9,9,9,10,10,10,11,11,None
Label IDs,-100,0,0,-100,0,0,5,-100,-100,6,...,5,-100,-100,-100,6,-100,-100,0,-100,-100
Labels,IGN,O,O,IGN,O,O,B-LOC,IGN,IGN,I-LOC,...,B-LOC,IGN,IGN,IGN,I-LOC,IGN,IGN,O,IGN,IGN


**(NOTE)** 부분단어 표현을 마스킹하기 위해 ID로 -100을 선택한 이유

파이토치에 있는 크로스 엔트로피 손실 클래스 `torch.nn.CrossEntropyLoss`의 `ignore_index` 속성 값이 -100 이기 때문에 

훈련하는 동안 이 인덱스는 무시됨. 따라서 연속된 부분단어에 해당하는 토큰을 무시하기 위해 이 인덱스를 사용함

레이블 ID와 토큰을 어떻게 정렬하는 지 살펴봄.

이제 이 로직을 감싼 함수를 하나 정의해서 전체 데이터 셋에 적용

In [44]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = xlmr_tokenizer(examples["tokens"], 
                                     truncation = True,
                                     is_split_into_words = True)
    labels = []
    
    for idx, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index = idx)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None or word_idx == previous_word_idx:
                label_ids.append(-100)
            else:
                label_ids.append(label[word_idx])
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

각 분할을 인코딩하는 데 필요한 모든 요소를 준비.

분할에 대해 반복 수행할 함수 생성

In [45]:
def encode_panx_dataset(corpus):
    return corpus.map(tokenize_and_align_labels, batched = True, remove_columns = ['langs', 'ner_tags', 'tokens'])

이 함수를 DatasetDict 객체에 적용하면 분할마다 인코딩된 Dataset 객체를 얻음 

이 함수를 사용해 독일어 말뭉치를 인코딩

In [46]:
panx_de_encoded = encode_panx_dataset(panx_ch["de"])

Map:   0%|          | 0/12580 [00:00<?, ? examples/s]

Map:   0%|          | 0/6290 [00:00<?, ? examples/s]

Map:   0%|          | 0/6290 [00:00<?, ? examples/s]

모델과 데이터셋을 준비했으니 이제 성능 지표를 정의

## 4.7 성능 측정

NER 모델 평가는 텍스트 분류 모델 평가와 비슷

일반적으로 정밀도(precision), 재현율(re-call), F1-score 결과를 보고함 

유일한 차이는 예측 하나를 정확하다고 판단하기 위해 한 개체명에 있는 모든 단어가 올바르게 예측되어야 한다는 점

다행히 이런 종류의 문제를 위해 라이브러리 seqeval(https://huggingface.co/spaces/evaluate-metric/seqeval / https://github.com/chakki-works/seqeval)이 고안됨

가령 NER 태그와 모델 예측 결과를 전달하면 seqeval의 classification_report() 함수를 통해 이런 지표를 계산함

